# Project 6

# Pipeline 2: `Search`

In [1]:
# importing modules
from os import chdir
import pandas as pd
import lib.database_module as db
import lib.encoding_module as enc
from lib.encoding_module import get_searchterm_vector, get_page_vector
import lib.wiki_module as wiki



In [2]:
from sklearn.externals import joblib
transformer = joblib.load('vectorizer.pkl')

## Parse: Encode search as document vector

In [3]:
search_doc = "Steve Jobs" 

In [4]:
# this returns a dictionary with search_doc string as the key

search_vector_doc = get_searchterm_vector(transformer, search_doc)
search_vector_doc

{'Steve Jobs': array([[  3.52379698e-03,   1.55156385e-03,  -2.22391882e-03,
           1.48923200e-04,  -1.56076045e-03,   3.55996343e-04,
           3.66544673e-05,  -5.92638807e-04,  -9.80722891e-04,
           4.88593667e-04,  -1.92905728e-03,  -4.04103430e-04,
          -4.09921068e-04,   1.40910127e-03,  -1.03500250e-03,
           2.42885848e-03,   1.18846424e-04,  -2.61224733e-03,
           4.18605032e-04,  -3.25036946e-03,  -7.41455414e-04,
           2.96052564e-03,   1.55751352e-03,   8.22861752e-04,
          -5.76304049e-04,  -1.78838303e-03,   3.61818629e-03,
           1.37083318e-03,  -6.58150609e-04,   2.24037865e-04,
          -7.92612114e-04,  -1.44483477e-03,  -2.19441767e-03,
          -3.53477853e-03,  -2.85772562e-03,  -8.87745579e-04,
           1.43593606e-03,  -9.05940353e-04,  -5.93635142e-04,
           3.39636792e-03,  -7.64174455e-04,  -8.88899052e-04,
          -3.36521482e-04,   8.57806795e-04,  -4.00816816e-04,
           1.56177825e-03,   3.89576296e-

## Mine: Select document vectors for all pages from database

In [5]:
db.connect_to_postgres()

Connected to server joshuacook.me.


(<connection object at 0x114ea3640; dsn: 'user=nate password=xxxxxxxxx largest heading center' host=joshuacook.me port=5432 dbname=project_6', closed: 0>,
 <cursor object at 0x114ccf338; closed: 0>)

In [6]:
page_vectors = db.select_all_page_vectors()

Connected to server joshuacook.me.


In [7]:
indices = [tup[0] for tup in page_vectors]
vectors = [tup[1] for tup in page_vectors]

In [8]:
# Column 0 is the pageid

page_vectors_df = pd.DataFrame(vectors, index=indices)
page_vectors_df.head()

0         1         2         3         4         5    \
4341789   0.092992 -0.040985  0.006952  0.000243  0.001681  0.004999   
48201744  0.021301  0.027559 -0.003917 -0.010877  0.013055  0.000934   
2514975   0.124983  0.153025  0.026264 -0.257191 -0.210183  0.074847   
35135520  0.191357 -0.039003 -0.006018  0.006389 -0.001106 -0.004748   
27303975  0.034292  0.039499 -0.006095 -0.015156  0.026112 -0.039749   

               6         7         8         9      ...          490  \
4341789   0.003430  0.003523 -0.005129 -0.001911    ...    -0.021003   
48201744  0.000128  0.010317  0.001444 -0.009792    ...    -0.055040   
2514975   0.003862 -0.009666 -0.065705 -0.280972    ...    -0.022703   
35135520  0.015829  0.005612  0.011846  0.028248    ...     0.000578   
27303975 -0.081440 -0.036546  0.005076  0.018881    ...     0.030056   

               491       492       493       494       495       496  \
4341789  -0.039528  0.027716  0.021033 -0.034768 -0.022294  0.031537   
48201744  0.000110  0.057817  0.005156  0.058477  0.008807  0.043527   
2514975   0.015894 -0.002476  0.004992 -0.006653 -0.017394 -0.008525   
35135520 -0.004204  0.030916  0.010484 -0.012454  0.026050  0.038744   
27303975 -0.026965 -0.002426  0.042282  0.024082  0.004498 -0.009517   

               497       498       499  
4341789  -0.009710 -0.010331  0.025944  
48201744 -0.011693  0.064738  0.051296  
2514975   0.017188 -0.001596  0.000120  
35135520  0.015332 -0.008053  0.009506  
27303975 -0.005315 -0.003784 -0.010202  

[5 rows x 500 columns]

In [9]:
lookup_df = pd.DataFrame(page_vectors_df.index)
lookup_df.head()

0
0   4341789
1  48201744
2   2514975
3  35135520
4  27303975

## Model: Find five most similar documents based on document vectors

In [10]:
# 2 ways to do this

# First -- cosine similarity

# Second -- NearestNeighbors

# going to use NearestNeighbors

In [11]:
from sklearn.neighbors import NearestNeighbors

this_NN = NearestNeighbors()

this_NN.fit(page_vectors_df)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [12]:
search_vector_doc['Steve Jobs']

array([[  3.52379698e-03,   1.55156385e-03,  -2.22391882e-03,
          1.48923200e-04,  -1.56076045e-03,   3.55996343e-04,
          3.66544673e-05,  -5.92638807e-04,  -9.80722891e-04,
          4.88593667e-04,  -1.92905728e-03,  -4.04103430e-04,
         -4.09921068e-04,   1.40910127e-03,  -1.03500250e-03,
          2.42885848e-03,   1.18846424e-04,  -2.61224733e-03,
          4.18605032e-04,  -3.25036946e-03,  -7.41455414e-04,
          2.96052564e-03,   1.55751352e-03,   8.22861752e-04,
         -5.76304049e-04,  -1.78838303e-03,   3.61818629e-03,
          1.37083318e-03,  -6.58150609e-04,   2.24037865e-04,
         -7.92612114e-04,  -1.44483477e-03,  -2.19441767e-03,
         -3.53477853e-03,  -2.85772562e-03,  -8.87745579e-04,
          1.43593606e-03,  -9.05940353e-04,  -5.93635142e-04,
          3.39636792e-03,  -7.64174455e-04,  -8.88899052e-04,
         -3.36521482e-04,   8.57806795e-04,  -4.00816816e-04,
          1.56177825e-03,   3.89576296e-03,  -1.73228543e-03,
        

In [13]:
results_NN = this_NN.kneighbors(search_vector_doc['Steve Jobs'])
results_NN

(array([[ 0.25545628,  0.3564603 ,  0.37011145,  0.37979901,  0.38626868]]),
 array([[1172,  247,  795,  483,  566]]))

In [14]:
results_NN_list = results_NN[1].tolist()
results_NN_list

[[1172, 247, 795, 483, 566]]

In [15]:
results_NN_list = results_NN_list[0]
results_NN_list

[1172, 247, 795, 483, 566]

In [16]:
vmask = [True if i in results_NN_list else False for i in lookup_df.index]
vmask

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [17]:
NN_pages = lookup_df[0][vmask]

In [18]:
NN_pages

247     18007329
483     13846997
566     38485157
795      2904185
1172     8181732
Name: 0, dtype: int64

In [19]:
pages = list(NN_pages.values)
pages

[18007329, 13846997, 38485157, 2904185, 8181732]

In [20]:
# first set of numbers in the array -- are the euclidiean distances and the second row are the page_ids

## Mine: Retrieve text for the five most similar documents

In [21]:
# Using the wiki module to retrieve the 5 most similar documents

for page in pages:
    wiki_response = wiki.query_page(str(page))
    print wiki_response['text']

Karate terms come almost entirely from Japanese. The following terms are not exclusive to karate. They appear during its study and practice, varying depending on style and school. Karate terms include: C Chito-ryu D Dan – Dojo G Gi – Goju-ryu - H Hajime – Heian K Karate – Kata – Kihon – Kohai – Kumite – Kyū O Osu R Rei S Senpai – Sensei – Shihan – Shotokan – Sōke W Waza Z Zanshin See also References External links
The Covin is a replica kit car of the Porsche 911 Turbo created by Tim Cook and Nick Vincent in the early 1980s. The name Covin came about from CO (Cook) and VIN (Vincent) giving us COVIN Performance Mouldings. Early models of the Covin were based on a shortened Beetle floorpan running gear but later used its own Covin chassis and VW Type 3 running gear. The company was sold in the 1990s to DAX and later moved to new owners GPC and was relocated to County Galway in Ireland where unfortunately up to now the Covin has not been produced again. The Covin came in 3 body styles all

In [23]:
# Using the database module to retrieve the pages

db.select_pages(pages, location='remote')

Connected to server joshuacook.me.


[(2904185,
  'Simit (Turkish: simit, Arabic: \xd8\xb3\xd9\x85\xd9\x8a\xd8\xb7\xe2\x80\x8e, translit. sam\xc4\xabt\xcb\xa4\xe2\x80\x8e, Persian: \xd8\xb3\xdb\x8c\xd9\x85\xdb\x8c\xd8\xaa\xe2\x80\x8e, translit',
  'Simit (Turkish: simit, Arabic: \xd8\xb3\xd9\x85\xd9\x8a\xd8\xb7\xe2\x80\x8e, translit. sam\xc4\xabt\xcb\xa4\xe2\x80\x8e, Persian: \xd8\xb3\xdb\x8c\xd9\x85\xdb\x8c\xd8\xaa\xe2\x80\x8e, translit. s\xc4\xabm\xc4\xabt\xcb\xa4\xe2\x80\x8e, Bulgarian: \xd1\x81\xd0\xb8\xd0\xbc\xd0\xb8\xd1\x82  simit ), gevrek (Turkish: gevrek, Bulgarian: \xd0\xb3\xd0\xb5\xd0\xb2\xd1\x80\xd0\xb5\xd0\xba, Macedonian: \xd1\x93\xd0\xb5\xd0\xb2\xd1\x80\xd0\xb5\xd0\xba, Serbian: \xd1\x92\xd0\xb5\xd0\xb2\xd1\x80\xd0\xb5\xd0\xba), bokegh (Armenian: \xd5\xa2\xd5\xb8\xd5\xaf\xd5\xa5\xd5\xb2), or koulouri (Greek: \xce\xba\xce\xbf\xcf\x85\xce\xbb\xce\xbf\xcf\x8d\xcf\x81\xce\xb9) is a circular bread, typically encrusted with sesame seeds or, less commonly, poppy, flax or sunflower seeds, found across the cuisines 